In [ ]:
!pip install ifcopenshell graphviz networkx

# Ifc Graph Generation Tool
By Aaron Neugebauer.

Generate a graph representing the correlations of objects in a IFC file.

In [ ]:
import ifcopenshell as ifc
import graphviz as viz
import networkx as nx

Basic configurations.

In [ ]:
ifc_file = "TestGERMAN_OBD.ifc"
node_color: str = "#2E0DA6"
edge_color: str = "#8373BF"
file_base_name: str = "ifc_graph"

Default graph drawing function using [graphviz](https://graphviz.org/).

In [ ]:
def graph_from_edges(edges, nodes=None, name_addition=None, save_dot: bool = True, save_svg: bool = True):
    """
    Creates a drawing of a graph defined by the edges.
    If no node set is given, it will be created from the edges.
    """
    if nodes is None:
        nodes = set()
        for u,v in edges:
            nodes.add(u)
            nodes.add(v)

    g = viz.Graph(engine="sfdp", format="svg", node_attr={"shape": "point", "color": node_color}, edge_attr={"color": edge_color})
    g.graph_attr["size"] = "auto"
    g.graph_attr["overlap"] = "false"
    g.graph_attr["outputorder"] = "nodesfirst"
    for v in nodes:
        g.node(str(v), label=None)
    for u,v in edges:
        g.edge(str(u), str(v))

    name_add: str = name_addition if name_addition is not None else ""
    if save_dot:
        g.save(file_base_name + name_add + ".dot")
    if save_svg:
        g.render(outfile=file_base_name + name_add + ".svg")

def digraph_from_edges(edges, edge_labels: dict=None, nodes=None, name_addition=None, save_dot: bool = True, save_svg: bool = True):
    """
    Creates a drawing of a directed graph defined by the given edges.
    If no node set is given, it will be created from the edges.
    """
    if nodes is None:
        nodes = set()
        for u,v in edges:
            nodes.add(u)
            nodes.add(v)

    g = viz.Digraph(engine="sfdp", format="svg", node_attr={"color": node_color}, edge_attr={"color": edge_color})
    g.graph_attr["size"] = "auto"
    g.graph_attr["overlap"] = "false"
    g.graph_attr["outputorder"] = "nodesfirst"
    for v in nodes:
        g.node(str(v))
    for u,v in edges:
        label = None if edge_labels is None else edge_labels.get((u,v), None)
        g.edge(str(u), str(v), label=label)

    name_add: str = name_addition if name_addition is not None else ""
    if save_dot:
        g.save(file_base_name + name_add + ".dot")
    if save_svg:
        g.render(outfile=file_base_name + name_add + ".svg")

## Building the Graph
We use a simple DFS algorithm to discover all the edges and nodes reachable from the root with the id '1' of the graph.

In [ ]:
def traverse(model, element, prev_nodes: set[int], prev_edges: set[(int, int)], directed: bool = False):
    id: int = element.id()
    if id in prev_nodes:
        return
    prev_nodes.add(id)
    
    for elem in set(model.traverse(element, max_levels=1)).union(model.get_inverse(element)):
        elem_id = elem.id()
        if elem_id in prev_nodes:
            continue
        
        if directed:
            prev_edges.add((id, elem_id))
            prev_edges.add((elem_id, id))
        else:
            if (id, elem_id) not in prev_edges and (elem_id, id) not in prev_edges:
                prev_edges.add((id, elem_id))
        traverse(model, elem, prev_nodes, prev_edges)

In [ ]:
nodes: set[int] = set()
edges: set[(int, int)] = set()
model = ifc.open(ifc_file)
traverse(model, model.by_id(1), nodes, edges)

## Visualization of the Graph

In [ ]:
graph_from_edges(edges, nodes=nodes)

## Finding a Path between two IDs
We build the graph in networkX and use the build in path finding tool.

First we need to build the directed graph.

In [ ]:
di_nodes: set[int] = set()
di_edges: set[(int, int)] = set()
edge_labels: dict[(int, int), str] = None
model = ifc.open(ifc_file)
traverse(model, model.by_id(1), di_nodes, di_edges, directed=True)

In [ ]:
graph = nx.DiGraph()
for u,v in di_edges:
    graph.add_edge(u,v)

Define the sources and destinations of the desired paths pairwise.

In [ ]:
sources = [703, 703]
targets = [25858, 23461]

Calculate the shortest path between the sources and targets pairwise.

In [ ]:
paths = [nx.shortest_path(graph, s, t) for s,t in zip(sources, targets)]
for path in paths:
    print(path)


Determin if the edegs traversed by the path are either from 'traverse' or 'get_inverse' form the IFC model. This defines their edge labels.

In [ ]:
edge_labels: dict[(int, int), str] = {}
for u, v in edges_paths:
    if v in [e.id() for e in model.traverse(model.by_id(u))]:
        edge_labels[(u, v)] = "traverse"
    else:
        edge_labels[(u, v)] = "get_inverse"

print(edge_labels)

Render the each path by itself.

In [ ]:
edge_sets: list[set[int]] = [set((path[i], path[i + 1]) for i in range(len(path) - 1)) for path in paths]
for i, es in enumerate(edge_sets):
    addition: str = "_" + str(sources[i]) + "-" + str(targets[i])
    digraph_from_edges(es, name_addition=addition, edge_labels=edge_labels)

Render all paths together.

In [ ]:
edges_paths: set[(int, int)] = set()
for path in paths:
    for i in range(len(path) - 1):
        edges_paths.add((path[i], path[i + 1]))

digraph_from_edges(edges_paths, name_addition="_joined_paths",  edge_labels=edge_labels)